In [ ]:
# default_exp core2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
import checkerboard_lattice_plots as tools
from matplotlib import pyplot as plt
from matplotlib import transforms
from matplotlib.collections import LineCollection
import matplotlib.colors as colors
import matplotlib as mpl
from scipy.interpolate import interp1d
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from scipy.optimize import brentq,newton

# Auxiliary functions

> API details.

In [ ]:
#export
def numbern(mu_, nn_, beta_, energies_):
    e_aux = np.copy(energies_)
    e_aux[beta_ * (e_aux - mu_) > 30] = 30 / beta_ + mu_
    ferm = 1 / (np.exp(beta_ * (e_aux - mu_)) + 1)
    return np.sum(ferm) - nn_

In [ ]:
#export
def chern(states, mx, my, band):
        
        J_kx = np.zeros((mx, my, mx, my), dtype=int)
        J_ky = np.zeros((mx, my, mx, my), dtype=int)
        for ix in range(0, mx):
            for iy in range(0, my):
                    J_kx[ix, iy, (ix+1)%mx, iy] = 1 
                    J_ky[ix, iy, ix, (iy+1)%my] = 1
        
        J_kx = np.reshape(J_kx, (mx*my,mx*my))
        J_kx = np.argwhere(J_kx!=0)
        
        J_ky = np.reshape(J_ky, (mx*my,mx*my))
        J_ky = np.argwhere(J_ky!=0)
    
        U_up = np.einsum('ijk,ilk->jlk', np.conjugate(states[:, 0:int(band), J_ky[:, 0]]), states[:, 0:int(band), J_ky[:, 1]])
        U_right = np.einsum('ijk,ilk->jlk', np.conjugate(states[:, 0:int(band), J_kx[:, 0]]), states[:, 0:int(band), J_kx[:, 1]])
    
        U_u = np.zeros(mx*my, dtype=complex)
        U_r = np.zeros(mx*my, dtype=complex)
        
        for i1 in range(0, mx*my):
            U_u[i1] = np.linalg.det(U_up[:, :, i1])
            U_r[i1] = np.linalg.det(U_right[:, :, i1])
        
        U_u = U_u/np.abs(U_u)
        U_r = U_r/np.abs(U_r)
        
        
        U = U_r[J_kx[:,0]] * U_u[J_kx[:,1]] * (U_r[J_ky[:,1]])**(-1) * (U_u[J_kx[:,0]])**(-1)
        
        F = np.log(U)
        chern_band = 1/(2*np.pi*1j)*np.sum(F)
        return chern_band, F